In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
import torch
import time
import pandas as pd, html
from user_info import get_user_info

In [2]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
                                            load_in_8bit=True, 
                                            torch_dtype=torch.float16,
                                            device_map='auto')


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
# model.to(device)

In [4]:
start_time = time.time()
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])
print(f'Time taken {time.time()-start_time}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Yes, I do! Here's a simple yet delicious homemade mayonnaise recipe:

Ingredients:
- 1 egg yolk
- 1 teaspoon Dijon mustard
- 1 cup vegetable oil
- 2 tablespoons white wine vinegar or fresh lemon juice
- Salt to taste

Instructions:
1. In a medium bowl, whisk together the egg yolk and mustard.
2. Drizzle in the oil very slowly, while constantly whisking, until all the oil is incorporated and the mayonnaise thickens.
3. Add the vinegar or lemon juice and whisk until the mixture emulsifies.
4. Season with salt to taste.

Note: It is important to add the oil very slowly and whisk constantly, to ensure that the oil gets emulsified into the egg yolk instead of forming into large droplets, otherwise the mayonnaise might not turn out as thic

In [19]:
es_output = get_user_info(2669146)
try:
    res_text = es_output['resume'][0]
except Exception as e:
    pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15477.14it/s]


In [27]:
jt = "Assistant Manager - Tax & Regulatory Services (3-4 yrs)"
rt = html.unescape(res_text)

In [52]:
input_prompt = f'''
<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
unstructured text of a user's resume, and the title of a job that they have applied to. Your task 
is to check if the resume contains work experience relevant to the job title being provided.
Respond with a JUST A SINGLE WORD answer "yes" or "no"[\INST]
This is the resume text:\n{{resume_text}}\n
This is the job title {{job_title}}\n
[INST]The answer is [\INST]: 
'''

In [53]:
input_prompt = input_prompt.format(resume_text=res_text,
                   job_title=jt)

In [54]:
model_input = tokenizer(input_prompt, return_tensors="pt").to("cuda")

In [55]:
start_time = time.time()
model.eval()
with torch.no_grad():
    full_document = tokenizer.decode(model.generate(**model_input, max_new_tokens=32)[0], skip_special_tokens=True)

print(f'Time taken : {time.time()-start_time}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken : 1.5987660884857178


In [56]:
print(full_document)


 [INST]You are an accurate AI agent working for a job platform. You will be given the raw 
unstructured text of a user's resume, and the title of a job that they have applied to. Your task 
is to check if the resume contains work experience relevant to the job title being provided.
Respond with a JUST A SINGLE WORD answer "yes" or "no"[\INST]
This is the resume text:
CURRICULAM-VITAE
MOHIT PUNDEER
Tel: - 9871506490, 8219496810
Mail: -Mohitpundir73@gmail.com
Mohitkasouli007@gmail.com
Career Summary: -
A professional having 10 years of experience in taxation and auditing across various industries skilled in Finance &amp;
Taxation, reporting and assurance, looking forward to leverage knowledge and experience into a dynamic role in
finance and taxation related information technologies in an organization of great repute.
Personality Traits: -
Exhibits strong initiative to ensure efficient and effective completion of task.
Team player with good organizational skills, strong accounting and s